In [2]:
import os
os.chdir("../")

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')

In [4]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path 
    unzip_data_dir: str
    STATUS_FILE: Path 
    all_schema: dict

In [17]:
from src.DS_project.constants import *
from src.DS_project.utils.common import *

In [18]:
class ConfigurationManager:
    def __init__(self , 
                 config_path =CONFIG_FILE_PATH , 
                 params_path =PARAMS_FILE_PATH , 
                 schema_path=SCHEMA_FILE_PATH):
        self.config_path = read_yaml(config_path)
        self.params_path = read_yaml(params_path)
        self.schema_path = read_yaml(schema_path)

        create_directories([self.config_path.artifacts_root])

    def get_data_validation(self) -> DataValidationConfig:
        config = self.config_path.data_validation
        schema = self.schema_path.COLUMNS
        create_directories([config.root_dir])

        data_validation = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir= config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )

        return data_validation


In [19]:
class DataValidation:
    def __init__(self , config=DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE , "a") as f:
                        f.write(f"Validation Status: {validation_status} for column: {col}\n")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE , "a") as f:
                        f.write(f"Validation Status: {validation_status} for column: {col}\n")

        except Exception as e:
            logger.exception(e)
            raise e

In [20]:
try:
    cm = ConfigurationManager()
    data_validation_config = cm.get_data_validation()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    logger.exception(e)
    raise e

2025-07-02 15:12:15,623: INFO: common: Yaml file: config\config.yaml is loaded successfully
2025-07-02 15:12:15,630: INFO: common: Yaml file: params.yaml is loaded successfully
2025-07-02 15:12:15,637: INFO: common: Yaml file: schema.yaml is loaded successfully
2025-07-02 15:12:15,640: INFO: common: Directory artifacts is created
2025-07-02 15:12:15,643: INFO: common: Directory artifacts/data_validation is created
